In [1]:
import numpy as np
import time
from numba import cuda

# ==================== GPU - Calcul parallèle des interactions ====================
@cuda.jit #customization kernel 
def calcul_interactions_particules_gpu(position_x, position_y, vitesse_x, vitesse_y, charges, nombre_total, pas_temps):
    index_particule = cuda.grid(1)
    if index_particule < nombre_total:
        somme_force_x = 0.0
        somme_force_y = 0.0

        for autre_particule in range(nombre_total):
            if index_particule != autre_particule:
                vecteur_position_diff_x = position_x[autre_particule] - position_x[index_particule]
                vecteur_position_diff_y = position_y[autre_particule] - position_y[index_particule]
                distance = (vecteur_position_diff_x ** 2 + vecteur_position_diff_y ** 2 + 1e-9) ** 0.5

                force = (charges[index_particule] * charges[autre_particule]) / (distance ** 2)
                somme_force_x += vecteur_position_diff_x / distance * force
                somme_force_y += vecteur_position_diff_y / distance * force

        vitesse_x[index_particule] += somme_force_x * pas_temps
        vitesse_y[index_particule] += somme_force_y * pas_temps
        position_x[index_particule] += vitesse_x[index_particule] * pas_temps
        position_y[index_particule] += vitesse_y[index_particule] * pas_temps

# ==================== CPU - Simulation séquentielle ====================
def calcul_interactions_particules_cpu(position_x, position_y, vitesse_x, vitesse_y, charges, nombre_total, pas_temps):
    for particule_courante in range(nombre_total):
        somme_force_x = 0.0
        somme_force_y = 0.0

        for autre_particule in range(nombre_total):
            if particule_courante != autre_particule:
                vecteur_position_diff_x = position_x[autre_particule] - position_x[particule_courante]
                vecteur_position_diff_y = position_y[autre_particule] - position_y[particule_courante]
                distance = np.sqrt(vecteur_position_diff_x ** 2 + vecteur_position_diff_y ** 2 + 1e-9)

                force = (charges[particule_courante] * charges[autre_particule]) / (distance ** 2)
                somme_force_x += vecteur_position_diff_x / distance * force
                somme_force_y += vecteur_position_diff_y / distance * force

        vitesse_x[particule_courante] += somme_force_x * pas_temps
        vitesse_y[particule_courante] += somme_force_y * pas_temps
        position_x[particule_courante] += vitesse_x[particule_courante] * pas_temps
        position_y[particule_courante] += vitesse_y[particule_courante] * pas_temps

# ==================== Paramètres de simulation ====================
nombre_particules = 3000
intervalle_temporel = 0.01

positions_x = np.random.rand(nombre_particules).astype(np.float32) * 100  # initialisation des variable 
positions_y = np.random.rand(nombre_particules).astype(np.float32) * 100
vitesses_x = np.zeros(nombre_particules, dtype=np.float32)
vitesses_y = np.zeros(nombre_particules, dtype=np.float32)
valeurs_charge = (np.random.rand(nombre_particules).astype(np.float32) - 0.5) * 2  # entre -1 et 1

# ==================== Simulation CPU ====================
debut_cpu = time.time()    
calcul_interactions_particules_cpu(
    positions_x.copy(), positions_y.copy(),
    vitesses_x.copy(), vitesses_y.copy(),
    valeurs_charge.copy(), nombre_particules, intervalle_temporel
)
fin_cpu = time.time()
print(f"🧠 Durée de calcul CPU : {fin_cpu - debut_cpu:.4f} secondes")

# ==================== Préparation et lancement GPU ====================
gpu_pos_x = cuda.to_device(positions_x) # Envoi des données vers le GPU avec la fonction to_device
gpu_pos_y = cuda.to_device(positions_y)
gpu_vit_x = cuda.to_device(vitesses_x)
gpu_vit_y = cuda.to_device(vitesses_y)
gpu_charges = cuda.to_device(valeurs_charge)

threads_par_bloc = 256
nombre_blocs = (nombre_particules + threads_par_bloc - 1) // threads_par_bloc  ## je détermines combien de threads CUDA vont s’exécuter, et comment ils sont organisés. 

debut_gpu = time.time() #exécutée par des milliers de threads CUDA en parallèle, chacun s’occupant d’une particule.
calcul_interactions_particules_gpu[nombre_blocs, threads_par_bloc]( 
    gpu_pos_x, gpu_pos_y,
    gpu_vit_x, gpu_vit_y,
    gpu_charges, nombre_particules, intervalle_temporel
)
cuda.synchronize()
fin_gpu = time.time()

resultat_positions_x = gpu_pos_x.copy_to_host() #je copies les résultats depuis la mémoire du GPU vers la mémoire principale (CPU),   #afin de pouvoir les afficher, sauvegarder, ou comparer.
resultat_positions_y = gpu_pos_y.copy_to_host()
resultat_vitesses_x = gpu_vit_x.copy_to_host()
resultat_vitesses_y = gpu_vit_y.copy_to_host()
print(f"⚡ Durée de calcul GPU : {fin_gpu - debut_gpu:.4f} secondes")


🧠 Durée de calcul CPU : 18.8959 secondes


/opt/conda/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 12 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


⚡ Durée de calcul GPU : 0.4475 secondes
